---
description: 只需替换 import 语句，就能用 Langfuse 版本的 OpenAI SDK 获得完整的可观测性
category: Integrations
---


# 示例手册：OpenAI 集成（Python）


这是一个示例手册，演示如何在 Python 项目中集成 Langfuse 与 OpenAI。

Langfuse 会记录每次模型调用的输入输出，帮助你排查问题并评估质量。

按照 [集成指南](https://langfuse.com/integrations/model-providers/openai-py) 将本集成添加到你的 OpenAI 项目中。


## 环境准备


该集成适用于 OpenAI SDK `>=0.27.8`；若要使用异步函数和流式输出，请确保 SDK 版本 `>=1.0.0`。


In [ ]:
%pip install langfuse openai --upgrade

In [ ]:
import os

# Langfuse SDK 会通过环境变量读取鉴权信息，此处填入你在 Langfuse 控制台 Project Settings 中生成的密钥
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."  # 公钥：用于识别项目
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."  # 私钥：用于签名请求，务必妥善保管
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧盟区域默认服务端
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 如果你的项目在美国区域，请改用此地址

# OpenAI SDK 也默认通过环境变量获取 API Key，此处示例填入占位符
os.environ["OPENAI_API_KEY"] = "sk-proj-..."


In [ ]:
# Langfuse 提供了对原生 OpenAI SDK 的封装，接口保持一致但会自动上报调用数据
# 只需替换原本的 `import openai`，其余代码可以保持不变
from langfuse.openai import openai


## 示例

### 文本聊天补全


In [ ]:
# 示例：发起一次标准的聊天补全请求
# Langfuse 会记录请求名称 (name)、模型 (model)、消息内容 (messages) 以及自定义元数据 (metadata)
completion = openai.chat.completions.create(
  name="test-chat",  # 追踪名称，可在 Langfuse 中快速定位这次调用
  model="gpt-4o",  # 指定使用的模型
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},  # system 消息：定义助手的身份/行为
      {"role": "user", "content": "1 + 1 = "}],  # user 消息：用户的真实输入
  temperature=0,  # 温度越低越稳定，适合需要确定答案的场景
  metadata={"someMetadataKey": "someValue"},  # 自定义元数据，会显示在 Langfuse 调试界面
)

# 如果需要查看模型响应，可使用 completion.choices[0].message.content


### 图像聊天补全

这是一个演示 OpenAI 视觉能力的简单示例。你可以在 `user` 消息中直接传入图片。


In [ ]:
# 示例：向模型发送图像输入
completion = openai.chat.completions.create(
  name="test-url-image",  # 追踪名称
  model="gpt-4o-mini",  # GPT-4o、GPT-4o mini、GPT-4 Turbo 均具备视觉能力
  messages=[
      {"role": "system", "content": "You are an AI trained to describe and interpret images. Describe the main objects and actions in the image."},  # 指定模型任务
      {"role": "user", "content": [
        {"type": "text", "text": "What’s depicted in this image?"},  # 文字提示
        {
          "type": "image_url",
          "image_url": {
            "url": "https://static.langfuse.com/langfuse-dev/langfuse-example-image.jpeg",  # 示例图片 URL，Langfuse 会记录该链接
          },
        },
      ],
    }
  ],
  temperature=0,  # 图像描述多为事实陈述，建议设置较低温度
  metadata={"someMetadataKey": "someValue"},  # 自定义元数据
)

# Langfuse 会将图片 URL 也记录在追踪详情中，方便复现问题


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。

![聊天补全截图](https://langfuse.com/images/docs/multi-modal-trace.png)


### 流式聊天补全

这是一个演示 OpenAI 流式输出能力的简单示例。


In [ ]:
# 示例：开启流式输出，实时获取模型返回的 Token
completion = openai.chat.completions.create(
  name="test-chat",  # 追踪名称
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "You are a professional comedian."},
      {"role": "user", "content": "Tell me a joke."}],
  temperature=0,  # 温度越高，笑话越发散；此处设为 0 方便演示
  metadata={"someMetadataKey": "someValue"},
  stream=True  # 打开流式模式后，API 会边生成边返回
)

# completion 变为一个生成器，依次产出增量内容；打印时记得取消换行
for chunk in completion:
  print(chunk.choices[0].delta.content, end="")


Why don't scientists trust atoms?

Because they make up everything!None

### 异步聊天补全

该示例使用 OpenAI 的异步客户端。Langfuse 配置可通过环境变量或 `openai` 模块上的属性传入。


In [ ]:
# 异步示例：Langfuse 同样兼容 OpenAI 的 Async 客户端
from langfuse.openai import AsyncOpenAI

async_client = AsyncOpenAI()  # 实例化异步客户端，将自动复用环境变量中的 Langfuse 配置


In [ ]:
# 在异步函数内调用聊天补全接口，需要使用 await 等待结果
completion = await async_client.chat.completions.create(
  name="test-chat",  # 为本次调用命名
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 100 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)

# Langfuse 会自动关联异步调用上下文，不需要额外配置


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。

![聊天补全仪表盘](https://langfuse.com/images/docs/openai-chat.png)


### 函数调用

该示例演示如何借助 Pydantic 构建函数模式。


In [ ]:
%pip install pydantic --upgrade

In [ ]:
from typing import List
from pydantic import BaseModel

# 定义函数调用返回值的数据结构，让模型生成结构化的 JSON
class StepByStepAIResponse(BaseModel):
    title: str  # 标题：例如“装机步骤”
    steps: List[str]  # 步骤列表：每个元素是一句描述

schema = StepByStepAIResponse.schema()  # 返回 JSON Schema，供 OpenAI 函数调用使用


In [ ]:
import json

# 示例：引导模型调用我们定义的函数，并返回结构化结果
response = openai.chat.completions.create(
    name="test-function",
    model="gpt-4o-0613",  # 支持函数调用的模型版本
    messages=[
       {"role": "user", "content": "Explain how to assemble a PC"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",  # 函数名称，需要与业务代码保持一致
          "description": "Get user answer in series of steps",  # 告诉模型函数的用途
          "parameters": StepByStepAIResponse.schema()  # Pydantic 自动生成的参数定义
        }
    ],
    function_call={"name": "get_answer_for_user_query"}  # 强制模型调用指定函数
)

# Langfuse 会记录函数调用的入参与出参，便于追踪
output = json.loads(response.choices[0].message.function_call.arguments)  # 将字符串反序列化为 Python 字典


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。

![函数调用示例](https://langfuse.com/images/docs/openai-function.png)



## Langfuse 功能（用户、标签、元数据、会话）

你可以在 OpenAI 请求中加入额外属性，以启用更多 Langfuse 功能。Langfuse 集成会自动解析这些字段。完整功能列表见 [文档](https://langfuse.com/integrations/model-providers/openai-py#custom-trace-properties)。


In [ ]:
# Langfuse 扩展字段示例：通过 metadata/tags/user_id/session_id 丰富追踪信息
completion_with_attributes = openai.chat.completions.create(
  name="test-chat-with-attributes",  # trace 名称，对应 Langfuse 中的 Trace.name
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 1 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},  # trace 元数据，适合记录业务上下文
  tags=["tag1", "tag2"],  # trace 标签，可用于 Langfuse 控制台筛选
  user_id="user1234",  # 业务用户 ID，让你在 Langfuse 中按用户聚合
  session_id="session1234",  # 会话 ID，用于区分不同对话/请求
)


示例追踪：https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/286c5c70-b077-4826-a493-36c510362a5a


## AzureOpenAI

该集成同样支持 `AzureOpenAI` 与 `AsyncAzureOpenAI` 类。


In [ ]:
# Azure OpenAI 示例：在环境中填入 Azure 侧的密钥与部署信息
AZURE_OPENAI_KEY=""  # Azure 提供的 API Key
AZURE_ENDPOINT=""  # 你的 Azure OpenAI Endpoint，例如 https://xxx.openai.azure.com/
AZURE_DEPLOYMENT_NAME="cookbook-gpt-4o-mini"  # 在 Azure Portal 中预先创建的部署名称


In [ ]:
# 将原生 AzureOpenAI 客户端替换为 Langfuse 提供的版本，自动接入观测能力
from langfuse.openai import AzureOpenAI


In [ ]:
# 实例化 AzureOpenAI 客户端；参数与官方 SDK 完全一致
client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    api_version="2023-03-15-preview",  # 指定 Azure API 版本
    azure_endpoint=AZURE_ENDPOINT
)


In [ ]:
# 使用 Azure 部署的模型发起请求，Langfuse 会照常记录
client.chat.completions.create(
  name="test-chat-azure-openai",
  model=AZURE_DEPLOYMENT_NAME,  # Azure 部署名称映射到模型
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 1 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)


示例追踪：https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/7ceb3ee3-0f2a-4f36-ad11-87ff636efd1e


## 将多次生成归并为单个 Trace

在实际应用中，往往需要多次调用 OpenAI。借助 `@observe()` 装饰器，可以把一次 API 调用中的所有 LLM 请求归入 Langfuse 中同一个 `trace`。


In [ ]:
from langfuse.openai import openai
from langfuse import observe

# @observe 装饰器会自动：
# 1. 为 main 函数创建一个顶层 trace
# 2. 捕获函数内部的所有 Langfuse/OpenAI 调用，并将它们串联为一个完整链路
@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：询问国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked."},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content  # 读取模型回复

    # 嵌套调用 2：请模型写一首关于首都的诗
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a poet. Create a poem about a city."},
          {"role": "user", "content": capital}],
      temperature=1,  # 提高温度，让诗歌更有创意
      max_tokens=200,  # 控制输出长度
    ).choices[0].message.content

    return poem

# 直接调用 main 函数，Langfuse 会自动生成 trace 并可在控制台查看链路
print(main("Bulgaria", "admin"))


In Sofia's embrace of time's gentle hand,  
Where ancient whispers in the cobblestones stand,  
The Vitosha's shadow kisses the town,  
As golden sunsets tie the day down.  

Streets sing with echoes of footsteps past,  
Where stories linger, and memories cast,  
Beneath the banyan sky so wide,  
Cultures and histories peacefully collide.  

The Alexander Nevsky, majestic and bold,  
A guardian of faith with domes of gold,  
Its silence speaks in volumes profound,  
In the heart of a city where old truths are found.  

The rose-laden gardens in Boris' park,  
Perfume the air as day turns dark,  
While laughter and life dance at night,  
Under Sofia's tapestry of starlit light.  

Markets bustle with the color of trade,  
Where lively exchanges and histories fade,  
A mosaic of tales in woven rhyme,  
Sofia stands timeless through passage of time.  




前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看完整链路。

![多次 OpenAI 调用的追踪图](https://langfuse.com/images/docs/openai-trace-grouped.png)


## 完整方案：与 Langfuse SDK 协同

`trace` 是 Langfuse 的核心对象，你可以为它附加丰富的元数据。详见 [Python SDK 文档](https://langfuse.com/docs/sdk/python#traces-1)。

自定义 trace 后可以实现以下能力：
- 自定义名称，用来区分不同类型的链路
- 以用户为粒度的追踪
- 通过版本与发布信息进行实验管理
- 保存自定义元数据


In [ ]:
from langfuse.openai import openai
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层 Langfuse 客户端，可在装饰器之外手动操作 trace

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：获取国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked."},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content

    # 嵌套调用 2：根据首都生成诗歌
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a poet. Create a poem about a city."},
          {"role": "user", "content": capital}],
      temperature=1,
      max_tokens=200,
    ).choices[0].message.content

    # 手动更新当前 trace 的属性，让仪表盘信息更完整
    langfuse.update_current_trace(
        name="City poem generator",  # 自定义 trace 名称
        session_id="1234",  # 业务会话 ID
        user_id=user_id,  # 业务用户 ID
        tags=["tag1", "tag2"],  # 标签，支持在 Langfuse 中搜索
        public=True,  # 是否允许分享 Trace 链接
        metadata = {"env": "development"}  # 自定义元数据，例如环境标记
    )

    return poem

# create_trace_id() 会生成一个可复用的追踪 ID，你也可以改为自己的业务 ID
trace_id = langfuse.create_trace_id()

# 通过关键字参数 `langfuse_observation_id` 将 trace_id 传递给装饰器，方便串联上下游调用
print(main("Bulgaria", "admin", langfuse_observation_id=trace_id))


In the cradle of Balkan hills, she lies,  
A gem under cerulean skies,  
Sofia, where the ancient whispers blend,  
With modern souls, as time extends.

Her heart beats with the rhythm of the past,  
Where cobblestones and new dreams cast,  
A tapestry of age and youth, entwined,  
In every corner, stories unsigned.

The Vitosha stands like a guardian old,  
Whose peaks in winter snow enfold,  
The city below, glowing warm and bright,  
Under the embrace of evening light.

St. Alexander’s domes in sunlight gleam,  
Golden crowns of a Byzantine dream,  
While beneath, a bustling world unfurls,  
In markets vast, where culture swirls.

Winding streets where whispers linger,  
Liberty echoes from corner to finger,  
In the shadow of Soviet grandiosity,  
Bulgaria’s spirit claims its clarity.

Cafés breathe tales in the aroma of brew,


## 以编程方式添加评分


你可以向 trace 添加 [评分](https://langfuse.com/docs/scores)，记录用户反馈或自动化评估结果。评分可用于在 Langfuse 中筛选追踪，并会显示在控制台中。详见评分文档。

评分通过 `trace_id` 与对应的 trace 关联。


In [ ]:
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层客户端，用于主流程之外的操作

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main():
    # 在装饰器内部，可随时获取当前 trace 的 ID
    trace_id = langfuse.get_current_trace_id()

    # TODO: 在此处编写你的业务逻辑，例如继续调用其他 API、处理用户输入等

    return "res", trace_id

# 执行被装饰的函数，Langfuse 会生成 trace
_, trace_id = main()

# 在 trace 上下文外部也可以继续操作，例如向这次 trace 添加评分
langfuse.create_score(
    trace_id=trace_id,  # 指定要打分的 trace
    name="my-score-name",  # 评分名称，用于区分不同指标
    value=1  # 分值，可以是布尔、整数、浮点数，视业务场景而定
)
